# Clickhouse Database Workflow
1. download github data in native clickhouse format (74.6 gb, ~10hours to download)
2. clickhouse server must be running
see: https://clickhouse.tech/docs/en/getting-started/install/
 >sudo service clickhouse-server start (may need sudo -u japple)
 >clickhouse-client

## Insert the database into clickhouse
3. create the db tables:
 >CREATE TABLE github_events ...
 see https://ghe.clickhouse.tech/
4. Insert the DB file into clickhouse <E:\Documents\Clickhouse Github data\github_events_v2.native.xz>
  (~2-300 gb? takes ~4 hours to insert)
5. run code here to connect to clickhouse client and manipulate data

Note the clickhouse driver (python) communicates with the clickhouse server via a native TCP/IP protocol that ships data as typed values; this will cause problems when INSERT-ing into a DB, however I don't see this as an issue

## Documentation
For better (& future) documentation see:
* https://stackoverflow.com/questions/9195455/how-to-document-a-method-with-parameters
* https://www.sphinx-doc.org/en/master/usage/restructuredtext/domains.html#python-signatures

## local Clickhouse DB stuffs
#### find the earliest DB instance query:
```sql
SELECT min(created_at) AS first_seen
FROM github_events
WHERE toYear(created_at) < 2012

┌──────────first_seen─┐
│ 2011-02-12 13:00:00 │
└─────────────────────┘
```
#### find the earliest 10 commits:

```sql
SELECT
    created_at,
    repo_name
FROM github_events
WHERE (toYear(created_at) < 2012) AND (toMonth(created_at) < 3)
ORDER BY created_at ASC
LIMIT 10

Query id: 4703822a-986f-4a0f-a592-c031c945da0d

┌──────────created_at─┬─repo_name────────────────────┐
│ 2011-02-12 13:00:00 │ projectblacklight/blacklight │
│ 2011-02-12 13:00:06 │ ezmobius/super-nginx         │
│ 2011-02-12 13:00:10 │ /                            │
│ 2011-02-12 13:00:10 │ Selenium2/Selenium2          │
│ 2011-02-12 13:00:17 │ appcelerator/webkit_titanium │
│ 2011-02-12 13:00:19 │ urzuae/spd                   │
│ 2011-02-12 13:00:22 │ arokem/nitime                │
│ 2011-02-12 13:00:23 │ jkbrooks/sample_app          │
│ 2011-02-12 13:00:25 │ networkx/networkx            │
│ 2011-02-12 13:00:27 │ spurious/clang-mirror        │
└─────────────────────┴──────────────────────────────┘
20 rows in set. Elapsed: 105.222 sec. Processed 2.78 billion rows, 18.30 GB (26.38 million rows/s., 173.93 MB/s.)
```

------------------------------------------------------
#### find the latest 10 commits

Useful to seeing when the last entry in the DB was

```sql
SELECT
    created_at,
    repo_name
FROM github_events
WHERE (toYear(created_at) >= 2020) AND (toMonth(created_at) > 11)
ORDER BY created_at DESC
LIMIT 20

Query id: 2d80f96a-6db0-451e-9a69-6143e5ca915c

┌──────────created_at─┬─repo_name───────────────────────────────┐
│ 2020-12-07 08:59:59 │ Technigo/project-labyrinth              │
│ 2020-12-07 08:59:59 │ HUPO-PSI/mzQC                           │
│ 2020-12-07 08:59:59 │ elrumo/macOS_Big_Sur_icons_replacements │
│ 2020-12-07 08:59:59 │ MetaCipher/sdl-2.0-textures             │
│ 2020-12-07 08:59:59 │ womega/Covid19_project                  │
│ 2020-12-07 08:59:59 │ theGreatWhiteShark/hydrogen             │
│ 2020-12-07 08:59:59 │ diaslais/projeto-ohana                  │
│ 2020-12-07 08:59:59 │ commit-b0t/commit-b0t                   │
│ 2020-12-07 08:59:59 │ TheGameCreators/AGK-Studio              │
│ 2020-12-07 08:59:59 │ shortland/May-Automation                │
│ 2020-12-07 08:59:59 │ randomperson190/ControlDeOrganicos      │
│ 2020-12-07 08:59:59 │ markgardie/WebApp                       │
│ 2020-12-07 08:59:59 │ Lombiq/Orchard.AngularJS                │
│ 2020-12-07 08:59:59 │ Hugobros3/chunkstories-api              │
│ 2020-12-07 08:59:59 │ BenCrespoDuke/HackDuke                  │
│ 2020-12-07 08:59:59 │ AndrewUsher/tvmaze-graphql-server       │
│ 2020-12-07 08:59:59 │ HUPO-PSI/mzQC                           │
│ 2020-12-07 08:59:59 │ brothersu/L2ch05                        │
│ 2020-12-07 08:59:59 │ becklabs/sensor-buoy                    │
│ 2020-12-07 08:59:59 │ aboodKh/cra-devops                      │
└─────────────────────┴─────────────────────────────────────────┘

20 rows in set. Elapsed: 46.208 sec. Processed 2.81 billion rows, 53.47 GB (60.71 million rows/s., 1.16 GB/s.)
```



In [ ]:
# # check the environment
# import sys; sys.prefix

# # check the jupyter directory
# import os
# os.getcwd()

# # to install clickhouse support
# !{sys.executable} -m pip install clickhouse-driver

In [2]:
import sys
sys.prefix

'/home/japple/localDev/clickhouse/ch-env'

In [4]:
import pandas as pd
import sys,time
import math
from datetime import datetime
from sqlalchemy import create_engine
from clickhouse_driver import Client
# dependencies
# >ipython-sql
# install by command prompt:
# >conda install -yc conda-forge ipython-sql
client = Client('localhost')

In [9]:
# Read CSV file into DataFrame df
# cmc_repos_forge.csv has been copied into this folder
# NaN is assigned to empty cells
dfs = pd.read_csv('cmc_repos_forge.csv', index_col=0)
df = dfs[['repo','forge']].copy()

In [5]:
# subset dataframes for testing
# use .copy() as slicing will not allow for assignment
# df10 = df.iloc[:10].copy()
# df33 = df.iloc[:33].copy()

# test to show connectivity
# expected output: [('github_events',)]
client.execute('SHOW TABLES FROM default')

[('github_events',)]

In [8]:
# show GitHub Events information
infoQuery='''
SELECT   parts.*,
    columns.uncompressed_size,
    columns.ratio
FROM(SELECT
        database,
        table,
        formatReadableSize(sum(data_uncompressed_bytes)) AS uncompressed_size,
        formatReadableSize(sum(data_compressed_bytes)) AS compressed_size,
        round(sum(data_compressed_bytes) / sum(data_uncompressed_bytes), 3) AS ratio
    FROM system.columns
    GROUP BY
        database,
        table
) AS columns
RIGHT JOIN ( SELECT
        database,
        table,
        sum(rows) AS rows,
        max(modification_time) AS latest_modification,
        formatReadableSize(sum(bytes)) AS disk_size,
        sum(bytes) AS bytes_size
    FROM system.parts
    WHERE active AND parts.database = 'default'
    GROUP BY
        database,
        table
) AS parts ON (columns.database = parts.database) AND (columns.table = parts.table)
ORDER BY parts.bytes_size DESC
'''
client.execute(infoQuery)

[('default',
  'github_events',
  5632203654,
  datetime.datetime(2022, 4, 8, 4, 11, 52),
  '430.17 GiB',
  461893064868,
  '2.36 TiB',
  0.177)]

In [10]:
# ----------------------------------------------------
# --   S Q L   Q U E R Y   S T R I N G S   -----------
# ----------------------------------------------------

# ----------------------------------------------------
# ----S T A R S --------------------------------------
# ----------------------------------------------------
stars_L = '''
SELECT count() 
FROM github_events 
WHERE (event_type = 'WatchEvent') AND repo_name =
'''

# ----------------------------------------------------
# ----F O R K S --------------------------------------
# ----------------------------------------------------
forks_L = '''
SELECT count()
FROM github_events 
WHERE (event_type = 'ForkEvent') AND repo_name =
'''

# ----------------------------------------------------
# ----A U T H O R S-----------------------------------
# ----------------------------------------------------
# Calculates a monthly average from previous 3 months
# excluding current month because it is in progress
# 
# modify for static clickhouse data which stops at 2020-12-07
# >>created_at >= dateSub(MONTH, 6,toStartOfMonth(now())) AND
# >>created_at < dateSub(MONTH, 3,toStartOfMonth(now()))
#
authors_L = '''
SELECT
    ROUND( SUM(authors) / COUNT(month), 2) AS average
FROM
(
    SELECT 
        uniq(actor_login) AS authors,
        toMonth(created_at) AS month,
        toYear(created_at) AS year
    FROM github_events
    WHERE event_type IN ('PullRequestEvent', 'IssuesEvent', 'IssueCommentEvent', 'PullRequestReviewCommentEvent') AND
        repo_name = 
'''
# have to go back to Oct-Dec 2020 in the static database
# this is 17-14 months ago from now
authors_R = '''AND
        created_at >= dateSub(MONTH, 3,toStartOfMonth(now())) AND
        created_at < toStartOfMonth(now())
        /*created_at >= dateSub(MONTH, 17,toStartOfMonth(now())) AND
        created_at < dateSub(MONTH, 14,toStartOfMonth(now()))*/
    GROUP BY month, year
    ORDER BY year DESC, month DESC
)'''

# ----------------------------------------------------
# ----C O M M I T S-----------------------------------
# ----------------------------------------------------
# Calculates a monthly average from previous 3 months
# excluding current month because it is in progress
#
# modify for static clickhouse data which stops at 2020-12-07:
# >>created_at >= dateSub(MONTH, 6,toStartOfMonth(now())) AND
# >>created_at < dateSub(MONTH, 3,toStartOfMonth(now()))
# 
# note: there will be moderate timezone discrepancies, especially 
#       when calculating near the first of the month
#
commits_L ='''
SELECT ROUND( SUM(sum_push_distinct) / COUNT(month), 2) AS average
FROM
(
    SELECT SUM(push_distinct_size) AS sum_push_distinct, 
        toMonth(created_at) AS month,
        toYear(created_at) AS year
    FROM github_events
    WHERE repo_name = 
'''

commits_R = '''
AND 
        event_type = 'PushEvent' AND
        created_at >= dateSub(MONTH, 3,toStartOfMonth(now())) AND
        created_at < toStartOfMonth(now())
        /*created_at >= dateSub(MONTH, 17,toStartOfMonth(now())) AND
        created_at < dateSub(MONTH, 14,toStartOfMonth(now()))*/
    GROUP BY month, year
    ORDER BY year DESC, month DESC
)'''

# ----------------------------------------------------
# ----C O M M E N T S---------------------------------
# ----------------------------------------------------
# Calculates a monthly average from previous 3 months
# excluding current month because it is in progress
#
# total COMMENTS includes all commenting activity
# any comments counts as activity and increase engagement
# there are 3 event_type comment events:
# >CommitCommentEvent
# >IssueCommentEvent
# >CommitCommentEvent
#
comments_L='''
SELECT ROUND(SUM(total) / COUNT(month), 2) AS average
FROM
(
    SELECT
        (uniqIf(comment_id, event_type = 'PullRequestReviewCommentEvent') + 
         uniqIf(comment_id, event_type = 'IssueCommentEvent')) + 
         uniqIf(comment_id, event_type = 'CommitCommentEvent') AS total,
        toMonth(created_at) AS month,
        toYear(created_at) AS year
    FROM github_events
    WHERE repo_name =  
'''

comments_R='''
    AND 
        /*(created_at >= (toStartOfMonth(now()) - toIntervalMonth(17)) ) AND 
        (created_at <  (toStartOfMonth(now()) - toIntervalMonth(14)) )*/
        (created_at >= (toStartOfMonth(now()) - toIntervalMonth(3)) ) AND 
        (created_at <  (toStartOfMonth(now())) )
    GROUP BY
        month,
        year
    ORDER BY
        year DESC,
        month DESC
)'''

# ----------------------------------------------------
# ----P U L L   R E Q U E S T S   O P E N E D---------
# ----------------------------------------------------
# Calculates a monthly average from previous 3 months
# excluding current month because it is in progress
#
PR_L='''
SELECT
    ROUND( SUM(opened) / COUNT(month), 2) AS average
FROM
(
    SELECT  
        SUM(action = 'opened') AS opened,
        toYear(created_at) AS year, 
        toMonth(created_at) AS month
    FROM github_events
    WHERE repo_name = 
'''

PR_R='''
    AND 
        (event_type = 'PullRequestEvent') AND 
        /*(created_at >= (toStartOfMonth(now()) - toIntervalMonth(17)) ) AND 
        (created_at <  (toStartOfMonth(now()) - toIntervalMonth(14)) )*/
        (created_at >= (toStartOfMonth(now()) - toIntervalMonth(3)) ) AND 
        (created_at <  (toStartOfMonth(now())) )
    GROUP BY
        month,
        year
    ORDER BY
        year DESC,
        month DESC
)'''

# -------------------------------------------
# --- A V E R A G E  C A L C U L A T I O N
# -------------------------------------------
# timestamp set to 2021-03-31 to allow for a year of activity
#
avg_issue_time_L = '''
WITH repo_name =
'''
avg_issue_time_R = '''
AS repo,
     sum(dateDiff('minute', toDateTime(opened), toDateTime(closed)))/60/24 AS total_days,
     round(dateDiff('second', toDateTime(opened), toDateTime(closed)),2)/60 AS mins_open, 
	 count() AS num_issues,
     (event_type = 'IssuesEvent' OR event_type = 'PullRequestEvent') AS event,
     created_at >= toDateTime('2021-03-31') AS created
SELECT  round( total_days / num_issues, 2) AS average_response_time_days
FROM
(
    SELECT *
    FROM
   (
        SELECT 	number,
		created_at AS opened
	FROM github_events 
	WHERE 	repo AND
		event AND
		action = 'opened' AND
		created
    ) AS t1
    INNER JOIN
    (
        SELECT 	number,
		created_at AS closed
	FROM github_events 
	WHERE 	repo AND
		event AND
		action = 'closed' AND
		created
    ) AS t2 USING (number)
)
WHERE mins_open > 5
'''

# -------------------------------------------
# --  M E D I A N    C A L C U L A T I O N
# -------------------------------------------
# timestamp set to 2021-03-31 to allow for a year of activity
#
med_issue_time_L = '''
WITH repo_name =
'''
med_issue_time_R = '''
AS repo,
     round(dateDiff('second', toDateTime(opened), toDateTime(closed)),2)/60 AS mins_open,
     round(dateDiff('second', toDateTime(opened), toDateTime(closed)),2)/60/60/24 AS days_open, 
     (event_type = 'IssuesEvent' OR event_type = 'PullRequestEvent') AS event,
     created_at >= toDateTime('2021-03-31') AS created
SELECT  round(medianDeterministic(days_open, 1),2) as median_response_time_days
FROM
(
    SELECT *
    FROM
   (
        SELECT 	number,
		created_at AS opened
	FROM github_events 
	WHERE 	repo AND
		event AND
		action = 'opened' AND
		created
    ) AS t1
    INNER JOIN
    (
        SELECT 	number,
		created_at AS closed
	FROM github_events 
	WHERE 	repo AND
		event AND
		action = 'closed' AND
		created
    ) AS t2 USING (number)
)
WHERE mins_open > 5
'''

# ---------------------------------------------
# --- L O N G E V I T Y  C A L C U L A T I O N
# ---------------------------------------------
# searches the repo for all activity by a contributor
# excludes those that step in 'one-time'
# such as starring or forking or leaving a comment and never returning
# this excludes devs that work on other projects 
# (only calculates from single repo)
#---------------------------------------
# view multiple author's days active
#---------------------------------------
'''
WITH dateDiff('day', toDateTime(earliest_seen), toDateTime(last_seen)) AS days_active
SELECT
    days_active,
    actor_login,
    last_seen
FROM
(
    SELECT
        MIN(created_at) AS earliest_seen,
        MAX(created_at) AS last_seen,
        days_active,
        actor_login
    FROM github_events
    WHERE repo_name = 'bitcoin/bitcoin'
    GROUP BY actor_login
    ORDER BY days_active DESC
)
WHERE days_active > 0
LIMIT 100
'''

# ---------------------------------------------
# calculate A V G.  D E V.  L O N G E V I T Y
# ---------------------------------------------
avg_longevity_L = '''
WITH dateDiff('day', toDateTime(earliest_seen), 
	toDateTime(last_seen)) AS days_active
SELECT ROUND((SUM(days_active) / count() ),2) AS avg_dev_days_active
FROM
(  SELECT MIN(created_at) AS earliest_seen, 
		MAX(created_at) AS last_seen, 
		days_active, 
		actor_login, 
		count()
   FROM github_events 
   WHERE repo_name = 
'''
avg_longevity_R = '''
GROUP by actor_login
   ORDER by days_active DESC
)
WHERE days_active > 0
'''

In [17]:
# ----------------------------------------------------
# ----T E S T   Q U E R I E S-------------------------
# ----------------------------------------------------
# some of my testing notes

query_test_noStars = '''
SELECT 
    count() 
FROM github_events 
WHERE event_type = 'WatchEvent' 
    AND repo_name =
'bitcoin/bitcoin'
'''

query2 = '''
SELECT 
    count() 
FROM github_events 
WHERE event_type = 'WatchEvent' 
    AND repo_name =
'HuobiGroup/huobi-eco-chain'
'''

repo = '''
'HuobiGroup/huobi-eco-chain' 
'''

QUERY_AUTHORS_TEST = '''
SELECT
    ROUND( SUM(authors) / COUNT(month), 2) AS average
FROM
(
    SELECT 
        uniq(actor_login) AS authors,
        toMonth(created_at) AS month,
        toYear(created_at) AS year
    FROM github_events
    WHERE event_type IN ('PullRequestEvent', 'IssuesEvent', 'IssueCommentEvent', 'PullRequestReviewCommentEvent') AND
        repo_name = 'bitcoin/bitcoin' AND
        created_at >= dateSub(MONTH, 3,toStartOfMonth(now())) AND
        created_at < toStartOfMonth(now())
    GROUP BY month, year
    ORDER BY year DESC, month DESC
)'''

QUERY_COMMITS_TEST = '''
SELECT ROUND( SUM(sum_push_distinct) / COUNT(month), 2) AS average
FROM
(
    SELECT SUM(push_distinct_size) AS sum_push_distinct, 
        toMonth(created_at) AS month,
        toYear(created_at) AS year
    FROM github_events
    WHERE repo_name = 'bitcoin/bitcoin' AND 
        event_type = 'PushEvent' AND
        /*created_at >= dateSub(MONTH, 3,toStartOfMonth(now())) AND
        created_at < toStartOfMonth(now())*/
        created_at >= dateSub(MONTH, 17,toStartOfMonth(now())) AND
        created_at < dateSub(MONTH, 14,toStartOfMonth(now()))
    GROUP BY month, year
    ORDER BY year DESC, month DESC
)
'''

# expected output here:
# [(nan,)]
query_test_zero='''
SELECT ROUND( SUM(sum_push_distinct) / COUNT(month), 2) AS average
FROM
(
    SELECT SUM(push_distinct_size) AS sum_push_distinct, 
        toMonth(created_at) AS month,
        toYear(created_at) AS year
    FROM github_events
    WHERE repo_name = 'Uniswap/uniswap-v2-core' AND 
        event_type = 'PushEvent' AND
        /*created_at >= dateSub(MONTH, 3,toStartOfMonth(now())) AND
        created_at < toStartOfMonth(now())*/
        created_at >= dateSub(MONTH, 6,toStartOfMonth(now())) AND
        created_at < dateSub(MONTH, 3,toStartOfMonth(now()))
    GROUP BY month, year
    ORDER BY year DESC, month DESC
)'''


# View distribution of comments by month for a single year
'''
SELECT 
    uniq(comment_id) AS total_comments,
    uniqIf(comment_id, event_type = 'PullRequestReviewCommentEvent') AS pr_comments,
    uniqIf(comment_id, event_type = 'IssueCommentEvent') AS issue_comments,
    uniqIf(comment_id, event_type = 'CommitCommentEvent') AS commit_comments,
    toMonth(created_at) AS month,
    toYear(created_at) AS year
FROM github_events
WHERE 
   repo_name = 'bitcoin/bitcoin' AND
   toYear(created_at) >= 2020
GROUP BY month, year
ORDER BY year DESC, month DESC
'''


# only Sept/Oct/Nov 2020 #
QUERY_COMMENTS_TEST='''
SELECT ROUND(SUM(total) / COUNT(month), 2) AS average
FROM
(
    SELECT
        (uniqIf(comment_id, event_type = 'PullRequestReviewCommentEvent') + 
         uniqIf(comment_id, event_type = 'IssueCommentEvent')) + 
         uniqIf(comment_id, event_type = 'CommitCommentEvent') AS total,
        toMonth(created_at) AS month,
        toYear(created_at) AS year
    FROM github_events
    WHERE (repo_name = 'bitcoin/bitcoin') AND 
          (created_at >= (toStartOfMonth(now()) - toIntervalMonth(17)) ) AND 
          (created_at <  (toStartOfMonth(now()) - toIntervalMonth(14)) )
    GROUP BY
        month,
        year
    ORDER BY
        year DESC,
        month DESC
)'''

# view all PR activity sorted into: opened, closed, reopened
'''
SELECT  COUNT() AS total,
    SUM(action = 'opened') AS opened,
    SUM(action = 'closed') AS closed,
    SUM(action = 'reopened') AS reopened,
    toYear(created_at) AS year, 
    toMonth(created_at) AS month
FROM github_events
WHERE repo_name = 'bitcoin/bitcoin' AND 
    toYear(created_at) >= '2019' AND 
    event_type = 'PullRequestEvent'
GROUP BY month, year
ORDER BY year DESC, month DESC
'''

# monthly average over previous 3 months
'''
SELECT ROUND(SUM(opened) / COUNT(month), 2) AS average
FROM
(
    SELECT
        SUM(action = 'opened') AS opened,
        toYear(created_at) AS year,
        toMonth(created_at) AS month
    FROM github_events
    WHERE (repo_name = 'bitcoin/bitcoin') AND 
          (event_type = 'PullRequestEvent') AND 
          (created_at >= (toStartOfMonth(now()) - toIntervalMonth(18)) ) AND 
          (created_at <  (toStartOfMonth(now()) - toIntervalMonth(15)) )
    GROUP BY
        month,
        year
    ORDER BY
        year DESC,
        month DESC
)'''

# average response time
# includes 'IssuesEvent' or 'PullRequestEvent'
avg_issue_time_test = '''
WITH repo_name = 'bitcoin/bitcoin' AS repo,
     sum(dateDiff('minute', toDateTime(opened), toDateTime(closed)))/60/24 AS total_days,
     round(dateDiff('second', toDateTime(opened), toDateTime(closed)),2)/60 AS mins_open, 
	 count() AS num_issues,
     (event_type = 'IssuesEvent' OR event_type = 'PullRequestEvent') AS event,
     created_at >= toDateTime('2019-11-01') AS created
SELECT  round( total_days / num_issues, 2) AS average_response_time_days
FROM
(
    SELECT *
    FROM
   (
        SELECT 	number,
		created_at AS opened
	FROM github_events 
	WHERE 	repo AND
		event AND
		action = 'opened' AND
		created
    ) AS t1
    INNER JOIN
    (
        SELECT 	number,
		created_at AS closed
	FROM github_events 
	WHERE 	repo AND
		event AND
		action = 'closed' AND
		created
    ) AS t2 USING (number)
)
WHERE mins_open > 5
'''

# median response time
# includes 'IssuesEvent' or 'PullRequestEvent'
med_issue_time_test = '''
WITH repo_name = 'bitcoin/bitcoin' AS repo,
     round(dateDiff('second', toDateTime(opened), toDateTime(closed)),2)/60 AS mins_open,
     round(dateDiff('second', toDateTime(opened), toDateTime(closed)),2)/60/60/24 AS days_open, 
     (event_type = 'IssuesEvent' OR event_type = 'PullRequestEvent') AS event,
     created_at >= toDateTime('2019-11-01') AS created
SELECT  round(medianDeterministic(days_open, 1),2) as median_response_time_days
FROM
(
    SELECT *
    FROM
   (
        SELECT 	number,
		created_at AS opened
	FROM github_events 
	WHERE 	repo AND
		event AND
		action = 'opened' AND
		created
    ) AS t1
    INNER JOIN
    (
        SELECT 	number,
		created_at AS closed
	FROM github_events 
	WHERE 	repo AND
		event AND
		action = 'closed' AND
		created
    ) AS t2 USING (number)
)
WHERE mins_open > 5
'''

# average developer days active in the repo
'''
WITH dateDiff('day', toDateTime(earliest_seen), toDateTime(last_seen)) AS days_active
SELECT ROUND(SUM(days_active) / count(), 2) AS avg_dev_days_active
FROM
(
    SELECT
        MIN(created_at) AS earliest_seen,
        MAX(created_at) AS last_seen,
        days_active,
        actor_login,
        count()
    FROM github_events
    WHERE repo_name = 'bitcoin/bitcoin'
    GROUP BY actor_login
    ORDER BY days_active DESC
)
WHERE days_active > 0
'''

In [11]:
# ---------------------------------------------------------------------
# -- R U N   Q U E R Y ------------------------------------------------
# ---------------------------------------------------------------------
#
def runQuery(column_name, df, query_L, query_R=''):
    """executes a Clickhouse query on a Pandas dataframe;
    the complete query is a string 'query_L + repo + query_R' where
    repo is the name of a github repository contained in df
    
    Keyword arguments:
    column_name -- the new column to be added to the dataframe, e.g. 'stars'
    df -- the pandas dataframe containing 'repo' and 'forge' columns
    query_L -- the SQL string preceeding the github repository name
    query_R -- the SQL string postceding the github repository name, this can be empty by default
    """
    start = datetime.now()
    num = 0
    num_u = 0
    for row in df.itertuples():
        # only github for now as client is connected to github_events DB
        if row.forge == 'github':
            repo = row.repo
            # skip the NaN repos
            if type(repo) == str:
                query = query_L + '\'' + repo + '\'' + query_R
                result = client.execute(query)
                num += 1

                # query returns a tuple of list elements accessible by [first list][first item]
                # empty list returns -1 meaning it has to be manually verified
                # average of zero returns a nan
                if len(result) == 0:
                    df.at[row.Index, column_name] = -1
                elif math.isnan(result[0][0]):
                    df.at[row.Index, column_name] = 0
                else: 
                    df.at[row.Index, column_name] = result[0][0]
                    num_u += 1

        # proof-of-life
        if num % 10 == 0: sys.stdout.write(".")
        sys.stdout.flush()
    
    # some log info
    now = datetime.now()
    elapsed = (now - start).total_seconds()
    output = column_name + ': ' + str(num) + ' repos queried and ' + str(num_u) + ' updated; Query took ' + str(round(elapsed,2)) + ' seconds.'
    print('\n' + output)
    
    return

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 600 entries, 1 to 1925
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   repo    443 non-null    object
 1   forge   436 non-null    object
dtypes: object(2)
memory usage: 14.1+ KB


In [13]:
# all available queries
runQuery('stars', df, stars_L)
runQuery('forks', df, forks_L)
runQuery('authors', df, authors_L, authors_R)
runQuery('commits', df, commits_L, commits_R)
runQuery('comments', df, comments_L, comments_R)
runQuery('PR_open', df, PR_L, PR_R)
runQuery('avg_resp_time', df, avg_issue_time_L, avg_issue_time_R)
runQuery('med_resp_time', df, med_issue_time_L, med_issue_time_R)
runQuery('avg_longevity_days', df, avg_longevity_L, avg_longevity_R)

.................................................................
stars: 430 repos queried and 430 updated; Query took 3.65 seconds.
.................................................................
forks: 430 repos queried and 430 updated; Query took 3.65 seconds.
.................................................................
authors: 430 repos queried and 178 updated; Query took 13.07 seconds.
.................................................................
commits: 430 repos queried and 160 updated; Query took 6.5 seconds.
.................................................................
comments: 430 repos queried and 219 updated; Query took 33.08 seconds.
.................................................................
PR_open: 430 repos queried and 164 updated; Query took 5.49 seconds.
.................................................................
avg_resp_time: 430 repos queried and 200 updated; Query took 17.52 seconds.
..................................................

In [14]:
df

,repo,forge,stars,forks,authors,commits,comments,PR_open,avg_resp_time,med_resp_time,avg_longevity_days
CMC_id,,,,,,,,,,,
1,bitcoin/bitcoin,github,72112.0,59013.0,140.33,317.67,2246.33,171.33,17.34,2.90,427.11
1027,ethereum/go-ethereum,github,40452.0,28052.0,139.33,48.33,419.00,74.67,13.06,1.48,282.49
825,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1839,binance-chain/bsc,github,1407.0,652.0,63.00,20.00,211.00,10.00,35.68,7.99,74.36
3408,centrehq/centre-tokens,github,296.0,298.0,4.00,0.00,6.33,1.50,11.79,6.41,217.58
...,...,...,...,...,...,...,...,...,...,...,...
2398,SelfKeyFoundation,github,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2090,ElKornacio/contracts-early,github,8.0,14.0,0.00,0.00,0.00,0.00,0.00,0.00,16.25
8028,mirror-protocol,github,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [15]:
# 'CMC_id' is the key to both dataframes, however 'repo', and 'forge' are duplicated
dfm = pd.merge(dfs, df, on = ['CMC_id','repo','forge'], how = 'outer')

# write SQL query update to clickhouse_queries.csv
dfm.to_csv('clickhouse_queries.csv', encoding='utf-8')

# timestamp
date_time = datetime.fromtimestamp(time.time())
# convert timestamp to string in dd-mm-yyyy HH:MM:SS
str_date_time = date_time.strftime("%d-%m-%Y-%H-%M-%S")
str_filename = 'clickhouse_queries_updated_' + str_date_time + '.csv'

# make a backup of clickhouse_queries.csv
dfm.to_csv(str_filename, encoding='utf-8')

print('Data written to: '+str_filename)

Data written to: clickhouse_queries_updated_11-04-2022-21-40-51.csv
